In [ ]:
# (Programming, data available here) The data includes volumes of 10 brain regions of 50 control 
# participants and 50 patients. Use the volumes of the 10 regions as input features and build
#   (a) a two-layer neural network classifier to predict the disease diagnosis. Use 5-fold inner cross-validation (as discussed in the class) to tune the learning rate hyperparameter. 
#   (b) train a linear SVM and use inner cross-validation to tune the hyperparameter C of the SVM. 
#   (c) For each classifier, report Accuracy, Precision, Recall, and AUC (average and standard deviation 
#       over the five folds), and compare the results in a table. (3’)1,400 × 563
# Note: For P5 and P6, please include source code, necessary comments and a description of models and
# procedures. Using Jupyter notebook is encouraged but other tools/languages are also acceptable.
